In [0]:
from pyspark.sql.functions import col
from delta.tables import *

bronze_prefix = "/Volumes/movies_catalog/bronze/bronze_zone_movies"
silver_prefix = "/Volumes/movies_catalog/silver/silver_zone_movies"

movies  = spark.read.format("delta").load(f"{bronze_prefix}/movies_top100")
ratings = spark.read.format("delta").load(f"{bronze_prefix}/movies_ratings")
revenue = spark.read.format("delta").load(f"{bronze_prefix}/movies_revenue")
casts   = spark.read.format("delta").load(f"{bronze_prefix}/movies_casts")
meta    = spark.read.format("delta").load(f"{bronze_prefix}/movies_meta")

join_key = "Title"

movies_ren = movies.toDF(*[
    "movies_" + c if c.lower() == "year" else c for c in movies.columns
])
ratings_ren = ratings.toDF(*[
    f"rt_{c}" if c != join_key else c for c in ratings.columns
])
revenue_ren = revenue.toDF(*[
    f"rev_{c}" if c != join_key else c for c in revenue.columns
])
casts_ren = casts.toDF(*[
    f"cast_{c}" if c != join_key else c for c in casts.columns
])
meta_ren = meta.toDF(*[
    f"meta_{c}" if c != join_key else c for c in meta.columns
])

silver = (
    movies_ren
        .join(ratings_ren, join_key, "left")
        .join(revenue_ren, join_key, "left")
        .join(casts_ren, join_key, "left")
        .join(meta_ren, join_key, "left")
)

if "movies_Year" in silver.columns:
    silver = silver.withColumnRenamed("movies_Year", "Year")
elif "year" in silver.columns:
    silver = silver.withColumnRenamed("year", "Year")

silver = silver.select(*dict.fromkeys(silver.columns))

(
    silver.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .partitionBy("Year")
        .save(f"{silver_prefix}/top100_combined")
)

print("✅ movido para Silver com sucesso")
